# Project 2

In [1]:
# ============================================
# 0. Import required libraries
# ============================================
import pandas as pd
import json
import plotly.express as px

In [3]:
# ============================================
# 1. Read raw housing data (Chinese columns)
# ============================================
# Replace the file name with your actual housing CSV
housing_raw = pd.read_csv("dataset.csv", encoding="utf-8")

# Take a quick look at the original columns
print(housing_raw.columns)
housing_raw.head()

Index(['标题', '居室数', '厅堂数', '卫生间数', '总面积', '建造年份', '居民楼总层数', '小区户数', '小区绿化率',
       '物业费用', '小区均价', '装修', '楼层分布', '物业类型', '产权性质', '产权年限', '房本年限', '区', '街道',
       '小区', '南', '南北', '近地铁', '车位充足', '户型方正', '多人关注', '有电梯', '价格'],
      dtype='object')


,标题,居室数,厅堂数,卫生间数,总面积,建造年份,居民楼总层数,小区户数,小区绿化率,物业费用,...,街道,小区,南,南北,近地铁,车位充足,户型方正,多人关注,有电梯,价格
0,扫码唻电联系房东，真的急疯了，这价格还卖不掉房东说要刀了我！,2,1,1,17,2018.0,11.0,50.0,30.0,0.40,...,虹桥,虹中路2弄小区,False,True,True,True,False,False,False,50
1,正南 3室2厅 万科海上传奇(一二期),3,2,1,97,2014.0,22.0,2336.0,45.0,3.00,...,御桥,万科海上传奇(一二期),True,False,True,False,False,False,False,910
2,虹浦新城南区 通透三房 预约看房,3,1,1,93,2007.0,11.0,1731.0,30.0,1.25,...,浦江,虹浦新城(南区),False,True,True,False,False,False,True,355
3,18号地铁口！进才 ，南北通透沉香苑，房东置换急 售,2,2,1,90,2013.0,11.0,971.0,35.0,1.20,...,航头,沉香苑一街坊,False,True,True,False,True,False,False,286
4,市区独栋别墅！送400平大花园！自带私家游泳池！位置不靠马路,5,3,3,300,2010.0,2.0,68.0,43.0,5.50,...,虹桥,虹梅别墅,True,False,True,True,False,False,False,4100


In [4]:
# ============================================
# 2. Rename Chinese columns to English
#    (adjust if your column names are different)
# ============================================
rename_dict = {
    "标题": "title",
    "居室数": "bedrooms",
    "厅堂数": "living_rooms",
    "卫生间数": "bathrooms",
    "总面积": "area_sqm",
    "建造年份": "year_built",
    "居民楼总层数": "building_floors",
    "小区户数": "community_num_households",
    "小区绿化率": "community_green_ratio",
    "物业费用": "property_fee_rmb_per_sqm",
    "小区均价": "community_avg_price_rmb_per_sqm",
    "装修": "decoration",
    "楼层分布": "floor_position",
    "物业类型": "property_type",
    "产权性质": "ownership_type",
    "产权年限": "property_rights_years",
    "房本年限": "years_since_purchase",
    "区": "district_cn",
    "街道": "subdistrict_cn",
    "小区": "community_cn",
    "南": "facing_south",
    "南北": "facing_south_north",
    "近地铁": "near_metro",
    "车位充足": "ample_parking",
    "户型方正": "regular_layout",
    "多人关注": "many_followers",
    "有电梯": "has_elevator",
    "价格": "total_price_10k_rmb"   # usually total price in 10k RMB
}

housing = housing_raw.rename(columns=rename_dict)

print(housing.columns)
housing.head()

Index(['title', 'bedrooms', 'living_rooms', 'bathrooms', 'area_sqm',
       'year_built', 'building_floors', 'community_num_households',
       'community_green_ratio', 'property_fee_rmb_per_sqm',
       'community_avg_price_rmb_per_sqm', 'decoration', 'floor_position',
       'property_type', 'ownership_type', 'property_rights_years',
       'years_since_purchase', 'district_cn', 'subdistrict_cn', 'community_cn',
       'facing_south', 'facing_south_north', 'near_metro', 'ample_parking',
       'regular_layout', 'many_followers', 'has_elevator',
       'total_price_10k_rmb'],
      dtype='object')


,title,bedrooms,living_rooms,bathrooms,area_sqm,year_built,building_floors,community_num_households,community_green_ratio,property_fee_rmb_per_sqm,...,subdistrict_cn,community_cn,facing_south,facing_south_north,near_metro,ample_parking,regular_layout,many_followers,has_elevator,total_price_10k_rmb
0,扫码唻电联系房东，真的急疯了，这价格还卖不掉房东说要刀了我！,2,1,1,17,2018.0,11.0,50.0,30.0,0.40,...,虹桥,虹中路2弄小区,False,True,True,True,False,False,False,50
1,正南 3室2厅 万科海上传奇(一二期),3,2,1,97,2014.0,22.0,2336.0,45.0,3.00,...,御桥,万科海上传奇(一二期),True,False,True,False,False,False,False,910
2,虹浦新城南区 通透三房 预约看房,3,1,1,93,2007.0,11.0,1731.0,30.0,1.25,...,浦江,虹浦新城(南区),False,True,True,False,False,False,True,355
3,18号地铁口！进才 ，南北通透沉香苑，房东置换急 售,2,2,1,90,2013.0,11.0,971.0,35.0,1.20,...,航头,沉香苑一街坊,False,True,True,False,True,False,False,286
4,市区独栋别墅！送400平大花园！自带私家游泳池！位置不靠马路,5,3,3,300,2010.0,2.0,68.0,43.0,5.50,...,虹桥,虹梅别墅,True,False,True,True,False,False,False,4100


In [ ]:
# ============================================
# 3. Create an English district column
#    so that it matches the population dataset
# ============================================
district_map = {
    "黄浦": "Huangpu",
    "徐汇": "Xuhui",
    "长宁": "Changning",
    "静安": "Jing'an",
    "普陀": "Putuo",
    "虹口": "Hongkou",
    "杨浦": "Yangpu",
    "闵行": "Minhang",
    "宝山": "Baoshan",
    "嘉定": "Jiading",
    "浦东": "Pudong New Area", 
    "金山": "Jinshan",
    "松江": "Songjiang",
    "青浦": "Qingpu",
    "奉贤": "Fengxian",
    "崇明": "Chongming",
    "上海周边": "Shanghai surroundings"  # will be dropped
}

housing["district"] = housing["district_cn"].map(district_map)

# Keep only listings that are inside the 16 districts (drop "Shanghai surroundings")
housing = housing[housing["district"].notna()]

housing[["district", "community_avg_price_rmb_per_sqm", "total_price_10k_rmb"]].head()


,district,community_avg_price_rmb_per_sqm,total_price_10k_rmb
0,Minhang,28445.0,50
1,Pudong New Area,103551.0,910
2,Minhang,34202.0,355
3,Pudong New Area,35224.0,286
4,Minhang,188277.0,4100


In [6]:
# ============================================
# 4. Compute average unit price per district
# ============================================
# Convert the unit price column to numeric just in case
housing["community_avg_price_rmb_per_sqm"] = pd.to_numeric(
    housing["community_avg_price_rmb_per_sqm"], errors="coerce"
)

price_by_dist = (
    housing
    .groupby("district", as_index=False)["community_avg_price_rmb_per_sqm"]
    .mean()
)

price_by_dist.rename(
    columns={"community_avg_price_rmb_per_sqm": "avg_unit_price_rmb_per_sqm_2024"},
    inplace=True
)

price_by_dist


,district,avg_unit_price_rmb_per_sqm_2024
0,Baoshan,44170.883324
1,Changning,79181.859089
2,Chongming,18641.060556
3,Fengxian,25851.761221
4,Hongkou,77053.640360
5,Huangpu,112514.436005
6,Jiading,38717.595484
7,Jing'an,83284.811588
8,Jinshan,17202.110835
9,Minhang,57078.919621


In [9]:
# ============================================
# 5. Read population data (already in English)
#    File structure (from your screenshot):
#    District_CN, District_EN, Population_10k,
#    Population_persons, Note
# ============================================
pop = pd.read_csv("shanghai_population_2024.csv")

pop.head()
print(pop.columns)


EmptyDataError: No columns to parse from file